In [9]:
from fractions import Fraction
from functools import lru_cache

# Funktion mit Memoization
memo = {}

def P(V, W, p, player='A'):
    """
    Berechnet die Gewinnwahrscheinlichkeit für Spieler A oder B rekursiv mit Memoization und exakten Brüchen.
    
    V: Liste mit Chips von Spieler A auf den Feldern
    W: Liste mit Chips von Spieler B auf den Feldern
    p: Liste mit Wahrscheinlichkeiten für jedes Feld als Fraction
    player: 'A' für Spieler A, 'B' für Spieler B
    """
    # Tuple für die aktuelle Spielsituation (damit es als Schlüssel im Memo genutzt werden kann)
    state = (tuple(V), tuple(W), player)
    
    # Memoization: Prüfen, ob das Ergebnis für diesen Zustand bereits berechnet wurde
    if state in memo:
        return memo[state]
    
    # Abbruchbedingungen
    if all(v == 0 for v in V) and any(w > 0 for w in W):
        result = 1 if player == 'A' else 0  # A gewinnt, wenn A keine Chips mehr hat
    elif all(w == 0 for w in W) and any(v > 0 for v in V):
        result = 1 if player == 'B' else 0  # B gewinnt, wenn B keine Chips mehr hat
    elif all(v == w for v, w in zip(V, W)):
        result = 0  # Unentschieden
    else:
        # Berechne die Summe der Wahrscheinlichkeiten für relevante Felder
        total_prob = sum(p[j] for j in range(len(V)) if V[j] >= 1 or W[j] >= 1)
        result = Fraction(0)

        # Rekursionsschleife über alle Felder
        for j in range(len(V)):
            if V[j] >= 1 and W[j] >= 1:  # Fall 1: A und B haben Chips auf Feld j
                V_new = list(V)
                W_new = list(W)
                V_new[j] -= 1
                W_new[j] -= 1
                result += p[j] / total_prob * P(V_new, W_new, p, player)
            elif W[j] >= 1 and V[j] == 0:  # Fall 2: Nur B hat Chips auf Feld j
                W_new = list(W)
                W_new[j] -= 1
                result += p[j] / total_prob * P(V, W_new, p, player)
            elif V[j] >= 1 and W[j] == 0:  # Fall 3: Nur A hat Chips auf Feld j
                V_new = list(V)
                V_new[j] -= 1
                result += p[j] / total_prob * P(V_new, W, p, player)

    # Zwischenergebnis speichern
    memo[state] = result
    return result

# Funktion für beide Spieler
def calculate_probabilities(U, V, p):
    """
    Berechnet die Gewinnwahrscheinlichkeiten für beide Spieler mit exakten Brüchen.
    
    U: Tupel oder Liste mit Chips von Spieler A
    V: Tupel oder Liste mit Chips von Spieler B
    p: Tupel oder Liste mit Wahrscheinlichkeiten für jedes Feld als Fraction
    """
    P_A_win = P(U, V, p, 'A')
    P_B_win = P(U, V, p, 'B')
    P_draw = 1 - P_A_win - P_B_win
    return P_A_win, P_B_win, P_draw

# Beispiel-Eingaben mit exakten Wahrscheinlichkeiten
U = (3, 2, 1)  # Chips von Spieler A
V = (4, 2, 0)  # Chips von Spieler B
p = (Fraction(1,2), Fraction(1, 3), Fraction(1, 6))  # Wahrscheinlichkeiten als Fraction

# Gewinnwahrscheinlichkeiten berechnen
P_A_win, P_B_win, P_draw = calculate_probabilities(U, V, p)

print(f"P_A({U},{V}) = {P_A_win}")
print(f"P_B({U},{V}) = {P_B_win}")
print(f"P_U({U},{V}) = {P_draw}")


P_A((3, 2, 1),(4, 2, 0)) = 1181921/2400000
P_B((3, 2, 1),(4, 2, 0)) = 65/256
P_U((3, 2, 1),(4, 2, 0)) = 9511/37500
